<a href="https://colab.research.google.com/github/kittipon-shine/Learning_Visaulization/blob/main/Visualization_cmkl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sc
import matplotlib.pyplot as plt
import altair as alt
from sklearn.datasets import load_iris,load_wine,load_iris

#0. Load data

In [ ]:
# https://www.datacamp.com/community/tutorials/categorical-data
# df_flights = pd.read_csv('https://raw.githubusercontent.com/ismayc/pnwflights14/master/data/flights.csv')

# https://pbpython.com/categorical-encoding.html
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]
df_cars = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )

In [ ]:
iris = load_iris()
# for new version!
# iris = load_iris(as_frame=True)
# df_iris = iris['frame']

In [ ]:
df_iris = pd.DataFrame(iris['data'], columns=iris['feature_names'])
df_iris['target'] = iris['target']
mapper_iris_target = {0:'setosa',1:'vesicolor',2:'virginica'}
df_iris['target_name'] = df_iris['target'].map(mapper_iris_target)

In [ ]:
df_iris.head()

#1. Univariate visualization

In [ ]:
df_iris_uni = df_iris['sepal length (cm)']

##1.1 Scatter plot

In [ ]:
# matplotlib
# plt.figure(figsize=(7,7))
plt.scatter(x=df_iris_uni.index, y=df_iris_uni)
plt.xlabel('index')
plt.ylabel('sepal length (cm)')

In [ ]:
# seaborn
sns.scatterplot(x=df_iris_uni.index, y=df_iris_uni)

In [ ]:
# altair
points = alt.Chart(df_iris_uni.reset_index()).mark_point().encode(
    x='index',
    y='sepal length (cm)',
)
points

In [ ]:
# interactive plot from altair
points.interactive()

##1.2 Histogram plot

In [ ]:
plt.hist(df_iris_uni)

In [ ]:
# matplotlib
# plt.figure(figsize=(7,7))
plt.hist(df_iris_uni,bins=[4,4.5,5,5.5,6,6.5,7,7.5,8],density=True,rwidth=0.9)
plt.xlabel('sepal length (cm)')
plt.ylabel('count')

In [ ]:
# seaborn
sns.displot(df_iris_uni,bins=[4,4.5,5,5.5,6,6.5,7,7.5,8],kde=True)

In [ ]:
# altair
alt.Chart(pd.DataFrame(df_iris_uni)).mark_bar().encode(
    alt.X("sepal length (cm):Q",bin=alt.Bin(maxbins=10)),
    alt.Y('count():Q', stack=None)
)

# 2. Bivariate visualization

##2.1 nominal & nominal


### Heatmap

In [ ]:
df_titanic = sns.load_dataset("titanic")
df_titanic.head()

In [ ]:
df_titanic_nom_nom = df_titanic.loc[:,['class','embark_town']]
df_titanic_nom_nom_pivot = (df_titanic_nom_nom
                            .value_counts()
                            .to_frame('count')
                            .pivot_table(index='class',columns='embark_town',values='count'))
df_titanic_nom_nom_pivot

In [ ]:
df_titanic_nom_nom_pivot_pct = df_titanic_nom_nom_pivot.divide(df_titanic_nom_nom_pivot.sum(axis='rows'),axis='columns')
df_titanic_nom_nom_pivot_pct

In [ ]:
# seaborn
sns.set(font_scale=1.25)
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,4))

sns.heatmap(df_titanic_nom_nom_pivot, annot=True, fmt="d",ax=ax1)
sns.heatmap(df_titanic_nom_nom_pivot_pct, annot=True, fmt=".03f",ax=ax2)

ax1.set_title("Number of customer",fontsize=20)
ax2.set_title("Percent count of customer for each embark_town",fontsize=20)

In [ ]:
# altair
heatmap = alt.Chart(df_titanic_nom_nom_pivot.unstack().to_frame('count').reset_index()).mark_rect().encode(
    x='embark_town:N',
    y='class:N',
    color='count:Q'
)

number = alt.Chart(df_titanic_nom_nom_pivot.unstack().to_frame('count').reset_index()).mark_text(fontSize=20).encode(
    x='embark_town:N',
    y='class:N',
    text='count:Q',
    color=alt.condition(
        alt.datum.count > 500,
        alt.value('white'),
        alt.value('black')
    )
)
(heatmap+number).properties(
    width=200,
    height=150
)

##2.2 nominal & quantitative




In [ ]:
df_titanic_nom_quan = df_titanic.loc[:,['class','fare']]
df_titanic_nom_quan

### Barplot 

In [ ]:
# barplot
df_titanic_nom_quan_bar = df_titanic_nom_quan.groupby('class')[['fare']].mean().reset_index()
df_titanic_nom_quan_bar = df_titanic_nom_quan_bar.rename(columns={'fare':'mean_fare'})
df_titanic_nom_quan_bar

In [ ]:
sns.set(font_scale=1.25)
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,6))

ax1.bar(x='class',height='mean_fare',data=df_titanic_nom_quan_bar)
ax1.set_xlabel('class')
ax1.set_ylabel('mean of fare')
ax1.set_title('Mean of fare for each class',fontsize=20)

ax2.barh(y='class',width='mean_fare',data=df_titanic_nom_quan_bar)
ax2.set_xlabel('mean of fare')
ax2.set_ylabel('class')
ax2.set_title('Mean of fare for each class',fontsize=20)

fig.tight_layout(pad=1.0)


In [ ]:
sns.set(font_scale=1.25)
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,6))

ax1.plot(['First', 'Second', 'Third'],df_titanic_nom_quan_bar['mean_fare'])
ax1.set_xlabel('class')
ax1.set_ylabel('mean of fare')
ax1.set_title('Mean of fare for each class',fontsize=20)

ax2.plot(['First', 'Second', 'Third'],df_titanic_nom_quan_bar['mean_fare'])
ax2.set_ylim(0,90)
ax2.set_xlabel('class')
ax2.set_ylabel('mean of fare')
ax2.set_title('Mean of fare for each class',fontsize=20)

fig.tight_layout(pad=1.0)


### About distribution plot

In [ ]:
# distplot with multiple distribution
sns.kdeplot(x='fare',hue='class',data=df_titanic_nom_quan[df_titanic_nom_quan['class']=='First'].query('fare<100'))
sns.kdeplot(x='fare',hue='class',data=df_titanic_nom_quan[df_titanic_nom_quan['class']=='Second'].query('fare<100'))
sns.kdeplot(x='fare',hue='class',data=df_titanic_nom_quan[df_titanic_nom_quan['class']=='Third'].query('fare<100'))

In [ ]:
# Boxplot
sns.boxplot(x='class',y='fare',data=df_titanic_nom_quan.query('fare<100'))

In [ ]:
# Stripplot
sns.set(font_scale=1.25)
fig,([ax1,ax2],[ax3,ax4]) = plt.subplots(2,2,figsize=(8,8))

sns.stripplot(x='class', y='fare', data=df_titanic_nom_quan,jitter=False,ax=ax1)
sns.stripplot(x='class', y='fare', data=df_titanic_nom_quan,jitter=True,ax=ax2)

sns.stripplot(x='target_name', y='sepal width (cm)', data=df_iris,jitter=False,ax=ax3)
sns.stripplot(x='target_name', y='sepal width (cm)', data=df_iris,jitter=True,ax=ax4)


ax1.set_title("Titanic without jittering",fontsize=15)
ax2.set_title("Titanic with jittering",fontsize=15)

ax3.set_title("Iris without jittering",fontsize=15)
ax4.set_title("Iris with jittering",fontsize=15)

fig.tight_layout(pad=1.0)

In [ ]:
# Swarmplot
sns.set(font_scale=1.25)
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,6))
sns.swarmplot(x='class', y='fare', data=df_titanic_nom_quan,ax=ax1)
sns.stripplot(x='target_name', y='sepal width (cm)', data=df_iris,jitter=True,ax=ax2)

ax1.set_title("Titanic dataset",fontsize=15)
ax2.set_title("Iris dataset",fontsize=15)


In [ ]:
df_titanic

##2.3 quantitative & quantitative

###Scatter plot

In [ ]:
df_iris.loc[:,['sepal length (cm)','sepal width (cm)']]

In [ ]:
# seaborn
sns.scatterplot(x='sepal length (cm)',y='sepal width (cm)',data=df_iris)
plt.xlabel('sepal length')
plt.ylabel('sepal width')

In [ ]:
# altair
scatter = alt.Chart(df_iris).mark_point().encode(
    x='sepal length (cm)',
    y='sepal width (cm)',
)
scatter.properties(
    width=400,
    height=400
)

#Multivariate data visualization

##3.1 Scatter plot

In [ ]:
sns.scatterplot(x='sepal length (cm)',y='sepal width (cm)',hue='target_name',data=df_iris)
plt.xlabel('sepal length')
plt.ylabel('sepal width')

In [ ]:
# altair
scatter = alt.Chart(df_iris).mark_point().encode(
    x='sepal length (cm)',
    y='sepal width (cm)',
    color = 'target_name',
)
scatter.properties(
    width=400,
    height=400
)

In [ ]:
# CAR DATASET

# https://www.datacamp.com/community/tutorials/categorical-data
# df_flights = pd.read_csv('https://raw.githubusercontent.com/ismayc/pnwflights14/master/data/flights.csv')

# https://pbpython.com/categorical-encoding.html
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]
df_cars = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )

In [ ]:
df_cars_tmp = df_cars[df_cars['make'].isin(['isuzu','toyota','honda','volvo'])]

In [ ]:
sns.set(font_scale=1)
sns.scatterplot(x=df_cars_tmp['horsepower'],
                y=df_cars_tmp['wheel_base'],
                legend=False)

In [ ]:
sns.set(font_scale=1)
sns.scatterplot(x=df_cars_tmp['horsepower'],
                y=df_cars_tmp['wheel_base'],
                hue=df_cars_tmp['make']
                )
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.set(font_scale=1)
sns.scatterplot(x=df_cars_tmp['horsepower'],
                y=df_cars_tmp['wheel_base'],
                hue=df_cars_tmp['make'],
                size = df_cars_tmp['price']
                )
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


In [ ]:
sns.set(font_scale=1.25)
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(21,7))

sns.scatterplot(x=df_cars['horsepower'],
                y=df_cars['wheel_base'],
                legend=False,
                ax=ax1)
sns.scatterplot(x=df_cars['horsepower'],
                y=df_cars['wheel_base'],
                hue=df_cars['make'],
                legend=False,
                ax=ax2
               )
ax3=sns.scatterplot(x=df_cars['horsepower'],
                y=df_cars['wheel_base'],
                hue=df_cars['make'],
                size = df_cars['price'],
                legend=False,
                ax=ax3
               )

ax1.set_title('2 dimensions')
ax2.set_title('3 dimensions')
ax3.set_title('4 dimensions')

In [ ]:
# altair
scatter = alt.Chart(df_cars).mark_point().encode(
    x='horsepower',
    y='wheel_base',
    color = 'make',
    size = 'price',
    tooltip=['horsepower','wheel_base','fuel_type','num_doors' ,'make', 'price']
)
scatter.properties(
    width=400,
    height=400
).interactive()

##3.2 Stack&multiple bar chart

In [ ]:
# preprocess
df_titanic_stack_bar = df_titanic.loc[:,['sex','embark_town']].value_counts().to_frame('count').reset_index()
df_titanic_stack_bar = df_titanic_stack_bar.pivot_table(index='embark_town',columns='sex').reset_index()
df_titanic_stack_bar.columns = ['embark_town','count_male','count_female']
df_titanic_stack_bar

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,6))

ax1.bar(x='embark_town', height = 'count_male', label='Men',data=df_titanic_stack_bar)
ax1.bar(x='embark_town', height = 'count_female', bottom='count_male',label='Women',data=df_titanic_stack_bar)
ax1.set_title('stack bar chart',fontsize=20)
ax1.set_xlabel('embark town')
ax1.set_ylabel('Count')
ax1.legend()

w = 0.4

town_list = list(df_titanic_stack_bar['embark_town'])
X_axis = np.arange(len(town_list))

ax2.bar(X_axis - w/2, height = 'count_male',width=w, label='Men',data=df_titanic_stack_bar)
ax2.bar(X_axis + w/2, height = 'count_female',width=w, label='Women',data=df_titanic_stack_bar)

ax2.set_xticks(X_axis)
ax2.set_xticklabels(town_list)

ax2.set_title('multiple bar chart',fontsize=20)
ax2.set_xlabel('embark town')
ax2.set_ylabel('Count')
ax2.legend()


In [ ]:
# altair
stack_bar_chart = alt.Chart(df_titanic).mark_bar().encode(
    x='embark_town',
    y='count(*):Q',
    color='sex:N'
).properties(
    width=400,
    height=400
)

multiple_bar_chart = alt.Chart(df_titanic).mark_bar().encode(
    x=alt.X('sex:O',axis=alt.Axis(title=None, labels=False)),
    y='count(*):Q',
    color='sex:N',
    column=alt.Column('embark_town:N',header=alt.Header(titleOrient='bottom', labelOrient='bottom'))
).properties(
    width=100,
    height=400
)

In [ ]:
(stack_bar_chart|multiple_bar_chart).resolve_scale(y='shared',color='shared') 

##3.3 Spider chart

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
iris = load_iris()

df_iris = pd.DataFrame(iris['data'], columns=iris['feature_names'])
df_iris['target'] = iris['target']
mapper_iris_target = {0:'setosa',1:'vesicolor',2:'virginica'}
df_iris['target_name'] = df_iris['target'].map(mapper_iris_target)


In [ ]:
fig = go.Figure()

for i in range(3):
  foo = df_iris[df_iris['target']==i]
  foo = foo.drop(columns=['target','target_name'])
  foo['tmp'] = foo.iloc[:,0]
  r = np.array(foo).reshape(1,len(foo)*len(foo.columns))[0]
  theta = (foo.columns.tolist()[:-1]+foo.columns.to_list()[:1])*len(foo)
    
  fig.add_trace(
      go.Scatterpolar(
          r=r,
          theta=theta,
          name=f"IRIS-{mapper_iris_target[i]}",
          showlegend=True
      )
  )

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 9]
        )
    ),
    title="IRIS Flower Variables According to Flower Categories"
)

fig.show()

In [ ]:
avg_iris  = df_iris.groupby("target").mean()

In [ ]:
fig = go.Figure()
for i in range(3):
  fig.add_trace(
      go.Scatterpolar(
          r=avg_iris.loc[i].values.tolist() + avg_iris.loc[i].values.tolist()[:1],
          theta=avg_iris.columns.tolist() + avg_iris.columns.tolist()[:1],
          name=f"IRIS-{mapper_iris_target[i]}",
          showlegend=True
          )
  )

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 7]
        )
    ),
    title="IRIS Flower Variables According to Flower Categories"
)

fig.show()

##3.4 Parallel coordinate plot

In [ ]:
cols = ['sepal length (cm)','petal width (cm)','sepal width (cm)','petal length (cm)']
pd.plotting.parallel_coordinates(frame = df_iris, cols= cols, class_column = "target_name", color=["lime", "tomato","dodgerblue"])
plt.xticks(rotation=45)

In [ ]:
cols = ['petal length (cm)','petal width (cm)','sepal length (cm)','sepal width (cm)']
pd.plotting.parallel_coordinates(frame = df_iris, cols= cols, class_column = "target_name", color=["lime", "tomato","dodgerblue"])
plt.xticks(rotation=45)

In [ ]:
df_iris.columns

In [ ]:
alt.Chart(df_iris).transform_window(
    index='count()'
).transform_fold(
    ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)','petal width (cm)']
).mark_line().encode(
    x='key:N',
    y='value:Q',
    color='target_name:N',
    detail='index:N',
    opacity=alt.value(0.5)
).properties(width=500)

##3.5 Small multiple

In [ ]:
sns.pairplot(df_iris.drop(columns='target'),hue='target_name')

In [ ]:
# pairplot using altair
alt.Chart(df_iris).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='target_name:N'
).properties(
    width=150,
    height=150
).repeat(
    row=['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)','petal width (cm)'],
    column=['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)','petal width (cm)'],
).interactive()

# Visual variable separability

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,7))

sns.scatterplot(x='sepal length (cm)',y='sepal width (cm)',style='target_name',data=df_iris, ax=ax1)
sns.scatterplot(x='sepal length (cm)',y='sepal width (cm)',hue='target_name',data=df_iris, ax=ax2)
